In [ ]:
!pip install tfa-nightly
!pip install efficientnet
#!pip install tensorflow -U

In [ ]:

import math
from kaggle_datasets import KaggleDatasets
import glob
import os
from datetime import datetime
import numpy as np
import pickle
from IPython.display import FileLink

import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_probability as tfp

from tensorflow.keras.layers import Conv2D, Dense, Activation, BatchNormalization, Dropout, Flatten, Layer
from tensorflow.keras.applications import densenet
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from efficientnet.tfkeras import EfficientNetB6 # use tf 2.3 instead

from tensorflow.python.keras.layers.preprocessing import image_preprocessing as image_ops

import matplotlib.pyplot as plt
import pickle
tfa.register_all(custom_kernels=False)
#tfa.register_all()

print(tf.__version__)


In [ ]:
"""
height, width = tf.constant(610, tf.int64), tf.constant(730,tf.int64)
rgb_mean = tf.constant([152.7203985,  115.66881524,  94.62871587])
rgb_stddev=tf.constant([73.70250614, 67.17878002, 61.07537483])
rot_angle=10./180*math.pi
shear_angle=10./180*math.pi
scale=0.9 # smaller -> zoom
pad_margin=0.1

padSize, feedSize = 820, 224
padSizef, feedSizef = float(padSize), float(feedSize)

def matrix(a,b,c,d):
    return tf.reshape(tf.stack([a,b,c,d],axis=1), [-1,2,2])

def decodePad(x):
    x=tf.io.read_file(x)
    img=tf.io.decode_jpeg(x)
    imgshape=tf.shape(img)
    h=tf.math.maximum(tf.math.minimum((padSize-imgshape[0])//2,imgshape[0]),0)
    w=tf.math.maximum(tf.math.minimum((padSize-imgshape[1])//2,imgshape[1]),0)
    
    img=tf.pad(img, tf.reshape(tf.stack([h,h,w,w,0,0]),[3,2]),"SYMMETRIC")
    imgshape=tf.shape(img)
    h=padSize-imgshape[0]
    w=padSize-imgshape[1]
    img=tf.pad(img, tf.reshape(tf.stack([h//2,h//2+h%2,w//2,w//2+w%2,0,0]),[3,2]),"CONSTANT",0)
    imgshape=tf.shape(img)
    return tf.cast(img, tf.float32)

def get_shape(x):
    x=tf.io.read_file(x)
    img=tf.io.decode_jpeg(x)
    shape=tf.shape(img)
    return tf.cast(shape, tf.float32)#tf.cast(shape[0], tf.float32),tf.cast(shape[1], tf.float32)

def afterBatchPreprocess(data):
    bs=tf.shape(data)#[0]
    #data=_parse_image_function(data)
    
    img=tf.map_fn(decodePad, data, dtype=tf.float32)#, fn_output_signature=tf.float32)
    #img=tf.cast(img, tf.float32)
    shape=tf.map_fn(get_shape,data,dtype=tf.float32)#,fn_output_signature=tf.float32)
    imgwidth=tf.squeeze(tf.slice(shape,[0,1],[-1,1]))
    imgheight=tf.squeeze(tf.slice(shape,[0,0],[-1,1]))

    #imgwidth=tf.cast(data['image/width'], tf.float32)
    #imgheight=tf.cast(data['image/height'], tf.float32)
    height = 610.#tf.cast(height, tf.float32)
    width = 730.#tf.cast(width, tf.float32)

    
    # shear -> rotate -> scale
    rot=tf.random.uniform(shape=bs, minval=-rot_angle, maxval=rot_angle)
    shear_x=tf.math.tan(tf.random.uniform(shape=bs, minval=-shear_angle, maxval=shear_angle))
    shear_y=tf.math.tan(tf.random.uniform(shape=bs, minval=-shear_angle, maxval=shear_angle))
    cropScaleX=tf.math.minimum(imgwidth,width)/feedSizef
    cropScaleY=tf.math.minimum(imgheight,height)/feedSizef
    scale_x=tf.random.uniform(shape=bs, minval=scale, maxval=1)*cropScaleX
    scale_y=tf.random.uniform(shape=bs, minval=scale, maxval=1)*cropScaleY
    center_x=feedSizef/2.#padSizef/2.
    center_y=feedSizef/2.#padSizef/2.
    
    diff_x=(imgwidth-feedSizef*scale_x)/2.
    diff_y=(imgheight-feedSizef*scale_y)/2.
    
    rdCropW=tf.math.maximum(diff_x,0)
    rdCropH=tf.math.maximum(diff_y,0)
    rdCropW=tf.random.uniform(bs,minval=-rdCropW, maxval=rdCropW)
    rdCropH=tf.random.uniform(bs,minval=-rdCropH, maxval=rdCropH)

    trans=tf.eye(2, batch_shape=bs)
    trans=tf.linalg.matmul(matrix(tf.repeat(1.,bs), shear_x,shear_y, tf.repeat(1.,bs)),trans)
    trans=tf.linalg.matmul(matrix(tf.math.cos(rot), tf.math.sin(rot),-tf.math.sin(rot), tf.math.cos(rot)),trans)
    coor=tf.ones([tf.squeeze(bs),2,1], tf.float32)*padSizef/2.
    coor+=tf.expand_dims(tf.stack([rdCropW,rdCropH],axis=1), axis=2)
    new_coor=tf.linalg.matmul(trans,coor)
    trans=tf.linalg.matmul(matrix(scale_x, tf.repeat(0.,bs),tf.repeat(0.,bs), scale_y),trans)

    rdCrop_coor=tf.expand_dims(tf.stack([diff_x+rdCropW,diff_y+rdCropH],axis=1), axis=2)
    shift_coor = tf.expand_dims((padSizef-tf.stack([imgwidth,imgheight],axis=1))/2., axis=2)
    transform=tf.reshape(tf.concat([trans, coor-new_coor+shift_coor+rdCrop_coor], axis=2),[tf.squeeze(bs), -1])
    #transform=tf.reshape(tf.concat([trans, tf.zeros([tf.squeeze(bs),2,1], tf.float32)], axis=2),[tf.squeeze(bs), -1]) 
    transform = tf.pad(transform ,[[0,0],[0,2]], 'CONSTANT',0)
    img=tfa.image.transform(img, transform, interpolation='BILINEAR',output_shape=[feedSize,feedSize])
    return img#, data["image/class/label"], tf.gather(weights,data["image/class/label"])

filename = glob.glob("/kaggle/input/landmark-retrieval-2020/train/0/0/0/*")[0:10]
image = tf.constant(filename)
before=tf.io.read_file(image[0])
before=tf.io.decode_jpeg(before)
preprocessed = afterBatchPreprocess(image)

n=10
f, axarr = plt.subplots(n,2,figsize=(20,50))
for i in range(n):
    before=tf.io.read_file(image[i])
    before=tf.io.decode_jpeg(before)
    axarr[i,0].imshow(before)
    axarr[i,1].imshow(preprocessed[i,...]/255.)

"""

In [ ]:
def _parse_image_function(example_proto):
  # Parse the input tf.Example proto using the dictionary above.
  return tf.io.parse_example(example_proto, {
    'image/height': tf.io.FixedLenFeature([], tf.int64),
    'image/width': tf.io.FixedLenFeature([], tf.int64),
    'image/channels': tf.io.FixedLenFeature([], tf.int64),
    #'weight': tf.io.FixedLenFeature([], tf.float32),
    'image/class/label': tf.io.FixedLenFeature([], tf.int64),
    'image/encoded': tf.io.FixedLenFeature([], tf.string),
    })




#610.   730
height, width = tf.constant(610, tf.float32), tf.constant(730,tf.float32)
rgb_mean = tf.constant([152.7203985,  115.66881524,  94.62871587])
rgb_stddev=tf.constant([73.70250614, 67.17878002, 61.07537483])
rot_angle=10./180*math.pi
shear_angle=10./180*math.pi
scale=0.9 # smaller -> zoom
pad_margin=0.1
padSize, feedSize = 820, 224
padSizef, feedSizef = float(padSize), float(feedSize)

def matrix(a,b,c,d):
    return tf.reshape(tf.stack([a,b,c,d],axis=1), [-1,2,2])

def decodeCropResize(data):
    """
    cropping size dilemma:
    bigger : more distortion more information
    smaller: information loss less distortion
    """

    data=_parse_image_function(data)

    img=tf.io.decode_jpeg(data['image/encoded'])
    img = tf.image.random_crop(img, size=[tf.math.minimum(height, data['image/height']), tf.math.minimum(width, data['image/width']),data['image/channels']])
    img = tf.image.convert_image_dtype(img, tf.float32)
    img= tf.image.resize(img, [224, 224],method='bicubic')

    return img, data["image/class/label"]#, data['weight']

def decodePad(x):
    #x=tf.io.read_file(x)
    img=tf.io.decode_jpeg(x)
    imgshape=tf.shape(img)
    h=tf.math.maximum(tf.math.minimum((padSize-imgshape[0])//2,imgshape[0]),0)
    w=tf.math.maximum(tf.math.minimum((padSize-imgshape[1])//2,imgshape[1]),0)
    
    img=tf.pad(img, tf.reshape(tf.stack([h,h,w,w,0,0]),[3,2]),"SYMMETRIC")
    imgshape=tf.shape(img)
    h=padSize-imgshape[0]
    w=padSize-imgshape[1]
    img=tf.pad(img, tf.reshape(tf.stack([h//2,h//2+h%2,w//2,w//2+w%2,0,0]),[3,2]),"CONSTANT",0)
    imgshape=tf.shape(img)
    return tf.cast(img, tf.float32)

def afterBatchPreprocess(data,weights):
    bs=tf.shape(data)#[0]
    data=_parse_image_function(data)
    
    img=tf.map_fn(decodePad, data['image/encoded'],dtype=tf.float32)#, fn_output_signature=tf.float32)
    img = (img-rgb_mean)/rgb_stddev
    #img = img/255
    imgwidth=tf.cast(data['image/width'], tf.float32)
    imgheight=tf.cast(data['image/height'], tf.float32)
    #tf.cast(height, tf.float32)
    #tf.cast(width, tf.float32)
    
    # shear -> rotate -> scale
    rot=tf.random.uniform(shape=bs, minval=-rot_angle, maxval=rot_angle)
    shear_x=tf.math.tan(tf.random.uniform(shape=bs, minval=-shear_angle, maxval=shear_angle))
    shear_y=tf.math.tan(tf.random.uniform(shape=bs, minval=-shear_angle, maxval=shear_angle))
    cropScaleX=tf.math.minimum(imgwidth,width)/feedSizef
    cropScaleY=tf.math.minimum(imgheight,height)/feedSizef
    scale_x=tf.random.uniform(shape=bs, minval=scale, maxval=1)*cropScaleX
    scale_y=tf.random.uniform(shape=bs, minval=scale, maxval=1)*cropScaleY

    
    diff_x=(imgwidth-feedSizef*scale_x)/2.
    diff_y=(imgheight-feedSizef*scale_y)/2.
    
    rdCropW=tf.math.maximum(diff_x,0)
    rdCropH=tf.math.maximum(diff_y,0)
    rdCropW=tf.random.uniform(bs,minval=-rdCropW, maxval=rdCropW)
    rdCropH=tf.random.uniform(bs,minval=-rdCropH, maxval=rdCropH)

    trans=tf.eye(2, batch_shape=bs)
    trans=tf.linalg.matmul(matrix(tf.repeat(1.,bs), shear_x,shear_y, tf.repeat(1.,bs)),trans)
    trans=tf.linalg.matmul(matrix(tf.math.cos(rot), tf.math.sin(rot),-tf.math.sin(rot), tf.math.cos(rot)),trans)
    coor=tf.ones([tf.squeeze(bs),2,1], tf.float32)*padSizef/2.
    coor+=tf.expand_dims(tf.stack([rdCropW,rdCropH],axis=1), axis=2)
    new_coor=tf.linalg.matmul(trans,coor)
    trans=tf.linalg.matmul(matrix(scale_x, tf.repeat(0.,bs),tf.repeat(0.,bs), scale_y),trans)

    rdCrop_coor=tf.expand_dims(tf.stack([diff_x+rdCropW,diff_y+rdCropH],axis=1), axis=2)
    shift_coor = tf.expand_dims((padSizef-tf.stack([imgwidth,imgheight],axis=1))/2., axis=2)
    transform=tf.reshape(tf.concat([trans, coor-new_coor+shift_coor+rdCrop_coor], axis=2),[tf.squeeze(bs), -1])
    transform = tf.pad(transform ,[[0,0],[0,2]], 'CONSTANT',0)
    img=tfa.image.transform(img, transform, interpolation='BILINEAR',output_shape=[feedSize,feedSize])
    img=tf.image.random_flip_left_right(img)
    img = tf.reshape(img, [-1,feedSize,feedSize,3])
    return img, data["image/class/label"], tf.gather(weights,data["image/class/label"])
    

def decodeCropResizeAugment(data, weights):
    """
    cropping size dilemma:
    bigger : more distortion more information
    smaller: information loss less distortion
    data is a scalar, this function doesn't handle batches
    """

    data=_parse_image_function(data)
    
    img=tf.io.decode_jpeg(data["image/encoded"])
    img = tf.image.random_crop(img, size=[tf.math.minimum(tf.cast(height, tf.int64), data['image/height']), tf.math.minimum(tf.cast(width, tf.int64), data['image/width']),data['image/channels']])
    #img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.cast(img, tf.float32)
    img= tf.image.resize(img, [feedSize, feedSize],method='bilinear')
    

    return img, data["image/class/label"], tf.gather(weights,data["image/class/label"])

def after_batch(img, label, weight ):
    img = (img-rgb_mean)/rgb_stddev
    #img = img/255
    img=tf.image.random_flip_left_right(img)
    
    #img=tf.image.random_hue(img, 0.2)
    #tf.image.random_saturation(img,0.7,2.)
    #tf.image.random_brightness(img, 0.1)
    
    img = tf.reshape(img, [-1,feedSize,feedSize,3])
    return img, label, weight

#resampler = tf.data.experimental.rejection_resample(lambda features, label: label, target_dist=tf.ones(counts.size, tf.float32)/counts.size)
#sampled_train_ds=croped_dataset.apply(resampler).map(lambda a,b:b)

def get_weights(data):
    data.prefetch(-1)
    n=81313
    weights=np.zeros(n)
    length =0
    for i in data:
        weights[i["image/class/label"]]+=1
        length+=1
        if length%100000 == 0: print(length)
    weights=length/float(n)/np.maximum(weights,1)#[length/float(n)/max(1,weight) for weight in weights]
    return weights, length


def load_dataset( bs):
    classweight = np.load("/kaggle/input/landmark-retrieval-2020-classweights/weights.npy")
    datasets = glob.glob("/kaggle/input/google*")#[0:1]
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    kd = KaggleDatasets()
    files=[]
    gcss=[]
    print("loading dataset")
    for i in datasets:
        gcs = kd.get_gcs_path(os.path.basename(i))
        files.append(tf.io.matching_files(gcs+"/records/*"))
        gcss.append(gcs)
        #print(gcs)
    print(gcss)
    files = tf.concat(files, axis=0)
    filenames = tf.data.Dataset.from_tensor_slices(files)
    train = tf.data.TFRecordDataset(filenames, num_parallel_reads=-1)#filenames.interleave(lambda x: tf.data.TFRecordDataset(x), cycle_length=-1, block_length=1, num_parallel_calls=-1)
    train = train.with_options(ignore_order)
    train=train.shuffle(10000).map(lambda x: decodeCropResizeAugment(x, tf.constant(classweight)),num_parallel_calls=-1).batch(bs,drop_remainder=True).map(after_batch,num_parallel_calls=-1).prefetch(-1).repeat()
    #train=train.shuffle(10000).batch(bs,drop_remainder=True).map(lambda x: afterBatchPreprocess(x, tf.constant(classweight)),num_parallel_calls=-1).prefetch(-1).repeat()

    print("dataset loaded")
    return train

i=0
ds = load_dataset(64)
c=4
r=2
for img,l,w in ds.take(1):
    f, axarr = plt.subplots(r,c,figsize=(18, 8))
    for i in range(r):
        for j in range(c):
            axarr[i,j].imshow((img[c*i+j,...]*rgb_stddev+rgb_mean)/255.)


In [ ]:



def sigmoid_crossentropy(z,x):
    return tf.math.maximum(x, 0) - x * z + tf.math.log(1 + tf.math.exp(-tf.math.abs(x)))

class ArcFace(tf.keras.losses.Loss):
    def __init__(self,m=0.4,s=30):
        super(ArcFace, self).__init__( reduction = tf.keras.losses.Reduction.NONE)
        self.m=m
        self.s=s
        self.scce=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,reduction = tf.keras.losses.Reduction.NONE)# it handles the softmax

    def call(self, y, x):
        """ softmxx version
        x should be equal normalized_W * normalized_embeding_x, thus, all values must be in [-1,1] so acos can be applied
        """

        """
        y=tf.cast(tf.reshape(y, [-1]), tf.int32)
        shape=tf.shape(x)
        select=tf.gather(x, y, axis=1, batch_dims=1)
        select=tf.math.acos(select)+m
        select=self.s*tf.math.cos(select)
        indices=tf.expand_dims(tf.range(shape[0]), axis=1)
        indices=tf.concat([indices, tf.expand_dims(y, axis=1)], axis=1)
        x=tf.tensor_scatter_nd_update(x,indices,select)"""
        
        #loss = tf.keras.losses.sparse_categorical_crossentropy(y,x, from_logits=True)
        
        return self.scce(y,x)
        #super().call(y,x)
        
        """ hierarchical version
        y and x are ragged tensor with the exact same shape, same number of element
        x are the nodes values of the path (float)
        y : 0 if left 1 if right
        
        rl=(y+1)/2
        loss = tf.math.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(rl, x)*tf.math.abs(y), axis=1)
        """
        #return loss
    def get_config(self):
        config = super(ArcFace, self).get_config()
        config.update({"m": self.m, "s": self.s})
        return config



class MeanAveragePrecision(tf.keras.metrics.Metric):

    def __init__(self, name="mean average precision", top_k=100, max_query=1000, **kwargs):
        super(MeanAveragePrecision, self).__init__(name=name, **kwargs)
        self.map = self.add_weight(name='map', initializer='zeros', dtype=tf.float64)
        self.max_query=max_query
        self.top_k=top_k

    def update_state(self, embeddings, labels, sample_weight=None):
        """def tokeep(to_remove, idx):
            to_keep=tf.gather(y,to_remove)# get the label
            to_keep=tf.reduce_all(idx!=to_keep,0)# mask
            return tf.reshape(tf.where(to_keep),[-1])# get index"""




        tf.debugging.assert_equal(tf.shape(embeddings)[0],tf.shape(labels)[0])
        y, idx, count =tf.unique_with_counts(labels,out_idx=tf.int64)

        self.map.assign(tf.cond(tf.math.reduce_all(count==1),lambda: tf.constant((1/0)*0, dtype=tf.float64), lambda: self.computenext(embeddings, labels)))


    def computenext(self, embeddings, labels):
                #@tf.function
        def threshold(data,value):
            """
            data: 1D tensor tf.int                       4           | |
            value: the integral to remove                3       |   | |                         (5 strokes are removed)
            returns the threshold value to clip          2       |   | |   value=5  ->  |   | |  returns 2
            Finds the horizontal line to clip.           1       | | | |                | | | |

            mean=tf.cast(tf.math.reduce_mean(data), tf.int32)
            step=2
            while tf.cond(mean>step*2, lambda: True, lambda: False):
                step*=2
            thre=step
            one_flag=False
            change_flag=False
            before=True
            ok=True

            while ok:# Dichotomy
                #step/=2
                if step!=1: step//=2
                else: one_flag=True
                if tf.cond(tf.math.reduce_sum(tf.math.maximum(data-thre,0))>value, lambda: True, lambda: False):
                    if not before:
                        change_flag=True
                    else:
                        change_flag=False
                    before=True
                else:
                    if before:
                        change_flag=True
                    else:
                        change_flag=False
                    before=False
                ok = not(one_flag and change_flag)
                thre+=step if before else -step

            return thre+1 if before else thre"""

            mean=tf.cast(tf.math.reduce_mean(data), tf.int64)
            step=tf.constant(2, tf.int64)
            step=tf.while_loop(lambda step: mean>step*2, lambda  step: step*2, [step])[0]#while tf.cond(mean>step*2, lambda: True, lambda: False):
            thre=tf.identity(step)
            one_flag, change_flag=tf.constant(False),tf.constant(False)
            before, ok=tf.constant(True),tf.constant(True)


            def whileloop(t,s,o,c,b,ok):
                def sup():
                    c = tf.math.logical_not(b)#tf.cond(not b, lambda: tf.constant(True), lambda: tf.constant(False))
                    return c, tf.constant(True)
                def inf():
                    c = tf.identity(b)#tf.cond(b, lambda: tf.constant(True), lambda: tf.constant(False))
                    return c, tf.constant(False)

                s,o=tf.cond(s!=1, lambda: (s//2,tf.constant(False)), lambda: (s, tf.constant(True)))
                c,b=tf.cond(tf.math.reduce_sum(tf.math.maximum(data-t,0))>value, sup, inf)
                ok=tf.math.logical_and(o,c)
                t+=tf.cond(before, lambda: s, lambda: -s)
                return [t,s,o,c,b,ok]

            thre,_,_,_,before,_ = tf.while_loop(lambda t,s,o,c,b,ok: ok, whileloop, [thre, step, one_flag, change_flag, before, ok])
            thre=tf.cond(before, lambda: thre+1, lambda: thre)
            return thre




        def take(data,classes,to_keep):
            """
            data contains unique values listed in classes
            to_keep defines how much we want to keep for each label
            e.g to_keep=[3,2,1]->we keep 3 element of label "0", 2 of label "1" and 1 of label "2"
            thus, indices of to_keep must match to indicies of classes
            returns indices of the removed part and the remaining part (they are a partition)
            """
            nclass=tf.shape(classes)[0]
            ldata=tf.shape(data)[0]
            mask=data==tf.expand_dims(classes,axis=1)
            data=tf.ragged.boolean_mask(tf.repeat(tf.expand_dims(tf.range(ldata),axis=0),repeats=nclass,axis=0),mask)
            mask=tf.ragged.range(data.row_lengths())<tf.expand_dims(to_keep,axis=1)
            remove=tf.ragged.boolean_mask(data,mask)
            keep=tf.where(tf.math.reduce_all(tf.expand_dims(remove.flat_values,axis=1)!=tf.range(ldata), axis=0))
            return remove.flat_values, tf.squeeze(keep)
                   # queries             database

        y, idx, count =tf.unique_with_counts(labels,out_idx=tf.int64)

        nclasses=tf.shape(y,out_type=tf.dtypes.int64)[0]
        ldata=tf.shape(labels,out_type=tf.dtypes.int64)[0]
        morethanone=tf.cast(tf.where(count>1,1,0), tf.int64)
        def moreClassesThanMaxQuery():
            lastsolos=tf.slice(tf.squeeze(tf.where(count==1)),[0],[self.max_query])
            lastsolos=tf.gather(lastsolos,[self.max_query-1])
            return tf.where(tf.range(ldata)<lastsolos,morethanone,0)
        def lessClassesThanMaxQuery():
            thre=threshold(count, tf.math.minimum(self.max_query-nclasses,tf.math.floordiv(ldata,2)+1))# find the value to clip
            return tf.math.maximum(tf.constant(0, tf.int64),count-thre)+morethanone

        splitToQueriesAndDatabase=tf.cond(tf.math.count_nonzero(morethanone)>self.max_query, lambda: moreClassesThanMaxQuery(), lambda: lessClassesThanMaxQuery())
        idx_queries, idx = take(labels, y, splitToQueriesAndDatabase)
        embeddings_d, labels_d = tf.gather(embeddings, idx), tf.gather(labels, idx)# update again
        embeddings_q, labels_q = tf.gather(embeddings, idx_queries), tf.gather(labels, idx_queries)
        y, idx, count =tf.unique_with_counts(labels_d)
        #label2count=tf.lookup.StaticHashTable(tf.lookup.KeyValueTensorInitializer(y,count),0)


        nd=tf.shape(labels_d,out_type=tf.int64)[0]
        nq=tf.shape(labels_q,out_type=tf.int64)[0]
        # sort k-nearest
        a=tf.expand_dims(embeddings_q,axis=1)-embeddings_d
        a=tf.math.reduce_euclidean_norm(a,axis=2)
        a=tf.argsort(a)
        a=tf.gather(labels_d,a)
        rel=a==tf.expand_dims(labels_q,axis=-1)# relevance



        """
        create range tensor from mask
        [[True, True, True],      [[1,2,3],
         [False,False,True],  ->   [0,0,1],
         [True, False,True]]       [1,1,2]]
        """
        rg=tf.RaggedTensor.to_tensor(tf.ragged.range(tf.ones(nq, tf.int64)*nd))
        idx=tf.pad(rg,[[0,0],[0,1]], "CONSTANT",constant_values=nd)
        mask=tf.pad(rel,[[0,0],[0,1]], "CONSTANT",constant_values=True)
        idx=tf.ragged.boolean_mask(idx,mask)
        tosub=tf.pad(rg,[[0,0],[1,0]], "CONSTANT",constant_values=0)
        mask=tf.pad(rel,[[0,0],[1,0]], "CONSTANT",constant_values=True)
        tosub=tf.ragged.boolean_mask(tosub,mask)
        torepeat=idx-tosub
        p=tf.ragged.map_flat_values(lambda rg, torepeat: tf.repeat(rg,torepeat, axis=0), tf.RaggedTensor.from_tensor(rg),tf.RaggedTensor.from_tensor(tf.RaggedTensor.to_tensor(torepeat,shape=tf.shape(rg))))

        p=tf.RaggedTensor.to_tensor(tf.cast(p,tf.float64)/tf.cast(tf.ragged.range(tf.ones(nq, tf.int64),tf.ones(nq, tf.int64)*(nd+1)),tf.float64))*tf.cast(rel,tf.float64)
        p=tf.cond(nd>self.top_k, lambda: tf.slice(p,tf.zeros(tf.rank(p),dtype=tf.int64),[-1, self.top_k]), lambda: p)

        get_query_count = tf.gather(count,tf.squeeze(tf.slice(tf.where(y==tf.expand_dims(labels_q,axis=1)),[0,1],[-1,-1])))
        p=tf.reduce_sum(p,axis=1)/tf.cast(tf.math.minimum(get_query_count,[self.top_k]),tf.float64)#top k
        p=tf.reduce_mean(p)
        return p#self.map.assign(p)


    def result(self):
        return self.map

    def reset_states(self):
        self.map.assign(0)

class AdamAccu(tf.keras.optimizers.Adam):
    def __init__(self, grads_shape, lr=0.001, accumulation=1, clip_norm=1., **kwargs):
        super(AdamAccu, self).__init__(lr=lr, **kwargs)
        self.accumulation=accumulation
        self.count=0
        self.gradients=[tf.zeros_like(this_var) for this_var in grads_shape]
        self.clip_norm= clip_norm
    def apply_gradients(self,grad_and_var):
        self.count+=1
        it=iter(zip(*grad_and_var))
        grads=next(it)
        var=next(it)
        self.gradients = [accu + grad for accu, grad in zip(self.gradients, grads)]
        if self.count%self.accumulation==0:
            self.count=0
            self.gradients = [tf.clip_by_norm(grad/self.accumulation , self.clip_norm) for grad in self.gradients]
            super(AdamAccu,self).apply_gradients(zip(self.gradients,var))
            self.gradients=[tf.zeros_like(this_var) for this_var in self.gradients]
            
    def get_config(self):
        config = super(AdamAccu, self).get_config()
        config.update({"accumulation": self.accumulation, "clip_norm":self.clip_norm,
                       'grads_shape': [tf.zeros_like(this_var) for this_var in self.gradients]})
        return config

In [ ]:
class ClassLayer(tf.keras.layers.Layer):
    def __init__(self, units=81313, **kwargs):
        super(ClassLayer, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight( name="center",
            shape=(input_shape[-1], self.units),
            initializer="he_normal",
            trainable=True,
        )
        #self.b = self.add_weight(name="center_bias",shape=(self.units,), initializer=tf.keras.initializers.Zeros(), trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w)# + self.b
    
    def get_config(self):
        #config = super(ClassLayer, self).get_config()
        #config.update({"units": self.units})
        return {"units": self.units}#config


class Densenet2vec(tf.keras.Model):
    inter=1024
    D=512
    s=10
    m=0
    n=81313
    input=224
    def __init__(self):
        """
        train: {symbol:code,...} (use huffman)
        validation: batched tf.data.Dataset  ((,224,224,3),())
        """
        super(Densenet2vec,self).__init__()

        #self.dense_net=densenet.DenseNet121(input_shape=(224,224,3), weights='../input/densenet-keras/DenseNet-BC-121-32-no-top.h5', include_top=False, pooling="avg")
        #self.dense_net=densenet.DenseNet169(input_shape=(224,224,3), weights=None if load_weights else "imagenet", include_top=False, pooling="avg")
        self.dense_net=EfficientNetB6(include_top=False,weights=None ,input_shape=(self.input,self.input,3),pooling='avg')

        for layer in self.dense_net.layers:
            layer.trainable = True

        self.dense1 = Dense(self.inter, activation=tf.keras.layers.LeakyReLU(), kernel_initializer="he_normal")
        #, kernel_regularizer=regularizers.l1_l2(0.01), activity_regularizer=regularizers.l2(0.01))(x)
        self.flatten=Flatten()
        self.bn = BatchNormalization(fused=None)
        #self.relu1 = Activation('relu')
        self.drop=Dropout(0.1)
        #self.dense2 = Dense(768)#, kernel_regularizer=regularizers.l1_l2(0.01), activity_regularizer=regularizers.l2(0.01))(x)
        #self.relu2 = Activation('relu')
        self.dense3 = Dense(self.D, kernel_initializer="he_normal")
        self.norm=tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
        self.bn_train = BatchNormalization(fused=None)
        self.classLayer = ClassLayer( self.n, dtype='float32')


        #if load_weights:
        self.initLayers()
        #self.load_weights(load_weights)

    def call(self, image):

        X=self.dense_net(image)
        X=self.flatten(X)
        X=self.dense1(X)
        X=self.bn(X)
        #X=self.relu1(X)
        #X=self.drop(X)
        X=self.dense3(X)
        #X=self.norm(X)


        return X

    def train_step(self, data):

        if len(data) == 3:
            x, y, sample_weight = data
        else:
            x, y = data
            sample_weight=1

        tf.debugging.check_numerics(x,"input is wrong")

        with tf.GradientTape() as tape:
            x = self.call(x)  # Forward pass
            tf.debugging.check_numerics(x,"call is wrong")
            x = self.bn_train(x)

            #x=self.relu1(x)
            #x = self.norm(x)*self.s
            x = self.classLayer(x)

            loss = self.compiled_loss(y, x, sample_weight=sample_weight, regularization_losses=self.losses)
            
            tf.debugging.check_numerics(loss,"loss1 is wrong")
            #if isinstance(self.optimizer,mixed_precision.LossScaleOptimizer):
                #loss=self.optimizer.get_scaled_loss(loss)
            #tf.debugging.check_numerics(loss,"loss2 is wrong")

        # Compute gradients
        
        gradients = tape.gradient(loss, self.trainable_variables)
        gradients = [(tf.clip_by_norm(grad, 1.0)) for grad in gradients]
        # Update weights
        #if isinstance(self.optimizer,mixed_precision.LossScaleOptimizer):
            #gradients=self.optimizer.get_unscaled_gradients(gradients)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        return {m.name: m.result() for m in self.metrics}#{"loss":loss}



    def test_step(self, data):

        if len(data) == 3:
            x, y, sample_weight = data
        else:
            x, y = data
            sample_weight=1
        tf.debugging.assert_integer(y)
        x=self(x)
        #self.compiled_metrics.update_state(embedding, label)
        x = self.bn_train(x)
        x = self.classLayer(x)
        #loss = self.compiled_loss(y, x,sample_weight=sample_weight, regularization_losses=self.losses)
        #loss = tf.nn.compute_average_loss(self.loss(y,x), sample_weight=sample_weight, global_batch_size=self.GBS)
        #return {**{"HAFL loss": loss}, **{m.name: m.result() for m in self.metrics}}
        return {"loss": loss}



    def metrics_valid(self, dataset):

        embeddings=tf.constant([], tf.float32)
        labels=tf.constant([], tf.int64)
        loss=tf.constant([], tf.float32)
        length=0
        for img, label in dataset:
            img=self(img)
            embeddings=tf.concat([embeddings, tf.reshape(img,[-1])], axis=0)
            labels=tf.concat([labels, label], axis=0)
            img = self.bn_train(img)
            img = self.norm(img)
            loss=tf.concat([loss, [self.compiled_loss(label, img, regularization_losses=self.losses)]],axis=0)
            length+=tf.shape(label)[0]

        embeddings=tf.reshape(embeddings,[length, self.D])#self.validation_length*self.validation_bs, self.D])
        shuffle=tf.random.shuffle(tf.range(length))#self.validation_length*self.validation_bs))
        embeddings, labels = tf.gather(embeddings,shuffle), tf.gather(labels,shuffle)
        self.compiled_metrics.update_state(embeddings, labels)
        return tf.math.reduce_mean(loss)
    
    def initLayers(self):
        x=tf.random.uniform([2,self.input,self.input,3])
        x=self(x)
        x = self.bn_train(x)
        x = self.classLayer(x)
    
    def get_config(self):
        config = super(Densenet2vec, self).get_config()
        config.update({"units": self.units})
        return config

In [ ]:

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)

    # instantiate a distribution strategy
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print(strategy.num_replicas_in_sync)
    BS=2*128* strategy.num_replicas_in_sync
    policy='mixed_bfloat16'
    lr=0.0001
except ValueError:
    if tf.test.is_gpu_available() :
        strategy = tf.distribute.MirroredStrategy()
        print('Mirror')
    else:
        strategy = tf.distribute.get_strategy()
        print('Default')
    BS=64
    policy='mixed_float16'
    lr=0.0001
    
length_train = 1514470

train= load_dataset(BS)

policy = mixed_precision.Policy(policy)
mixed_precision.set_policy(policy)


In [ ]:




class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self):#, dataset):
        super(CustomCallback,self).__init__()
        #self.model=model
        #self.dataset_valid=dataset
        #logdir = "logs/retrivial/" + datetime.now().strftime("%Y%m%d-%H%M%S")
        #self.file_writer = tf.summary.create_file_writer(logdir + "/metrics")
        #self.file_writer.set_as_default()
    #def on_epoch_end(self, epoch, logs=None):
    #def on_test_end(self, logs=None):
        #print("on_test_end callback")

    def on_epoch_end(self, epoch, logs=None):
        print("Saving weights")
        self.model.save_weights('efn2vec.h5', overwrite=True)
        with open('optimizer.pkl', 'wb') as f:
            pickle.dump(self.model.optimizer.get_weights(), f)
        
        #FileLink(r'densenet2vec.h5')
        #print("\nvalidation loss: {}".format(self.model.metrics_valid(self.dataset_valid)))
        #for m in self.model.metrics:
            #print("{} : {}".format(m.name,m.result()) )
            #tf.summary.scalar("{}".format(m.name), data=m.result(), step=epoch)
            
    #def on_batch_end(self, epoch, logs=None):
        #print(self.model.optimizer.get_weights()[1][0,0,0,0:3])

class WarmRestart():
  def __init__(self, max, min, cycle):
    self.max=max
    self.min=min
    self.cycle=cycle
    self.count=0
    self.decay=0.9
  def __call__(self,epoch, lr):
    self.count+=1
    if self.count%self.cycle==0:
      self.count=0
      self.max*=self.decay
      self.min*=self.decay
    lr = self.min+(self.max-self.min)/2.*(1+tf.math.cos(self.count*3.14159/self.cycle).numpy())
    print("count: {}\tlr: {}".format(self.count, lr))
    return lr

schedule = tf.keras.callbacks.LearningRateScheduler(WarmRestart(max=0.0005, min=0.00001, cycle=10).__call__)

#model.evaluate(tf_traindev_database.batch(128, drop_remainder=False))#traindev_images,traindev_labels))
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='efn2vec.h5',
                                                 save_best_only=False, save_weights_only=False, verbose=1)
#logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
#tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs, profile_batch = '2,50')

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, verbose=1)#, min_lr=0.0000001)


# """

# """
def buildFit(path, lr=0.0001,  epochs=10, callbacks=None):
    with open(path+'/optimizer.pkl', 'rb') as f:
        weight_values = pickle.load(f)
    with strategy.scope():

        model=Densenet2vec()#'/kaggle/input/notebook3eb84c5ce9/efn2vec.h5')
        model.load_weights(path+'/efn2vec.h5')
        tw = model.trainable_weights
        
        optimizer = tf.keras.optimizers.Adam(lr)#, clipnorm=1.
        #optimizer = AdamAccu(grads_shape=tw, lr=lr, accumulation=2)
        #optimizer = optimizer = mixed_precision.LossScaleOptimizer(optimizer, loss_scale='dynamic')
        
        optimizer._create_all_weights(tw)
        #optimizer.set_weights(weight_values)
        
        model.compile(
            optimizer=optimizer,
            loss=ArcFace()
            #,metrics=[MeanAveragePrecision()],run_eagerly=False
            #,experimental_steps_per_execution=length_train//BS # tf 2.3
            )
        
    #print(model.optimizer.get_weights()[1])
        model.optimizer.set_weights(weight_values)
    #print(model.optimizer.get_weights()[1])
    #print(weight_values[1][0,0,0,0:3])

    history=model.fit(train,
                  epochs=epochs, verbose=2, steps_per_epoch=length_train//BS
                 #,validation_data=dev #,validation_steps=
                 ,callbacks=callbacks
                 )

                 
    return history

buildFit(path="/kaggle/input/notebook3eb84c5ce9", lr=lr, epochs=12, callbacks=[CustomCallback(),reduce_lr])
#buildFit(path="/kaggle/input/landmark-retrieval-2020-classweights", lr=lr, epochs=12, callbacks=[CustomCallback(),reduce_lr])

In [ ]:
a=tf.ragged.constant([[1,2,3],[4,5]])
for x in a:
    print(x)


            
            
            
